In [1]:
import numpy as np
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data_utils
import torch
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
import pickle
#import ot
from abc import ABC, abstractmethod
import pickle
from statistic.wasserstein_distance import WassersteinDistance, WassersteinDistanceDeviation
def save_pkl(variable, name):
    name = name + '.pkl'
    output = open(name, 'wb')
    pickle.dump(variable, output)
    output.close()

In [2]:
curves=pd.read_pickle('data/curves_2_wind_100.pkl')
clusters_new=pd.read_pickle('data/clust_fin.pkl')

In [3]:
curves_tensors=[]
for curve in range (len(curves)):
  curves_tensors.append(torch.from_numpy(curves[curve]))

max_shape=np.max(np.unique(list(map(len, curves))))
new_curves=[]
for i in range(len(curves)):
    new=np.zeros((max_shape,3))
    new[:curves[i].shape[0],:3]=curves[i]
    new_curves.append(new)

In [4]:
tensor_set=torch.stack([torch.Tensor(i) for i in new_curves[:-1]])
data_set=data_utils.TensorDataset(tensor_set,)
train_loader=data_utils.DataLoader(data_set,batch_size=16,shuffle=True)

In [5]:
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)
      
class View(nn.Module):
    def __init__(self, *shape):
        super(View, self).__init__()
        self.shape = shape
    def forward(self, input):
        return input.view(input.size(0),*self.shape)


In [6]:
dimZ = 200 # Considering face reconstruction task, which size of representation seems reasonable?
batch_size=1

class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        def init_weights(m):
            if type(m) == nn.Linear:
                torch.nn.init.xavier_uniform_(m.weight)
                m.bias.data.fill_(0.0)
        
        self.encoder=nn.Sequential()
        self.encoder.add_module('linear1',nn.Linear(1*2236*3,500))
        self.encoder.add_module('relu_1',nn.ReLU())
        
        
        self.encoder.add_module('linear2',nn.Linear(500,100))
        self.encoder.add_module('active1',nn.Sigmoid())
        self.encoder.apply(init_weights)
    def forward(self, x):

      latent_code =self.encoder(x)


      return latent_code

In [8]:

device = torch.device('cuda')
# criterion = torch.nn.MSELoss()
# autoencoder =Encoder().cuda()
# optimizer = optim.Adam(autoencoder.parameters(),lr=1e-3)

In [9]:
import ot
from scipy import linalg
dist=WassersteinDistance(2)

In [10]:
image_progress=[]
loss_history=[]
for i in range(50):
    print(i)
    autoencoder.train(True)
    loss_epoch=[]
    for x_batch,y_batch in tqdm(train_loader):
        results=[]
        for batch in x_batch:
          x_batch_new=batch.reshape(1,-1)
          x_batch_new=Variable(x_batch_new).cuda()
          res=autoencoder(x_batch_new)
          results.append(res)

        a=np.random.randint(0,len(x_batch),4)
        euclide=torch.norm(results[a[0]]-results[a[1]],p=2)
        euclide_1=torch.norm(results[a[2]]-results[a[3]],p=2)
        distance=torch.tensor(dist(x_batch[a[0]],x_batch[a[1]])).cuda()
        distance_1=torch.tensor(dist(x_batch[a[2]],x_batch[a[3]])).cuda()
        if clusters[0]==clusters[1]:

        loss_1=criterion(euclide,distance)
        loss_2=criterion(euclide_1,distance_1)
        
        loss=loss_1+loss_2

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        loss_epoch.append(loss.data.cpu().numpy())
    if i%5==0:
      torch.save(autoencoder, 'model_simple_auto'+str(i)+'.ckpt')
      torch.save(autoencoder.state_dict(), 'params_model_simple_auto'+str(i)+'.ckpt')

    autoencoder.train(False)    

    loss_history.append(np.mean(loss_epoch))
    print(np.mean(loss_epoch))
    
  
    plt.figure(figsize=(8, 6))
    plt.title("Training loss decay")
    plt.xlabel("epoch")
    plt.ylabel("loss")
    plt.semilogy(loss_history, label = 'training loss')
    plt.show()